In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import time
import numpy as np

nltk.download('punkt_tab')
nltk.download('stopwords')
import pandas as pd
import re
import nltk

df = pd.read_csv("Listofstartups.csv", encoding='ISO-8859-1')
df.head()


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/hitmonlee/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hitmonlee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Incubation Center,Name of the startup,Location of company,Sector,Company profile
0,SIIC IIT KANPUR,E-Trainer Analytics Wizard Pvt ltd,New Delhi,Fit-Tech,"Fitbuddy, a flagship brand under E-Trainer Ana..."
1,SIIC IITK,Invariance Automation Private Limited,"Kanpur, UP",Industrial Automation,Currently we are developing the infrastructure...
2,"SIIC, IIT Kanpur",Neoperk Technologies Pvt. Ltd.,"Mumbai, Maharashtra",Agri-Tech - Soil Testing,Neoperk makes large scale soil testing extreme...
3,SIIC IIT KANPUR,WeRehab Technologies Pvt Ltd,Nagpur,Health tech,WeRehab Technologies Pvt Ltd. Is a startup fou...
4,IIT Kanpur,Arthavedika Tech Pvt Ltd,"Noida, Uttar Pradesh",Fintech,Arthavedika Tech Pvt Ltd is a DIPP certified t...


In [2]:
def preprocess_text(text: str) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)
    
    # remove stopwords
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
    text = " ".join(tokens)
    text = text.lower().strip()
    
    return text


In [3]:
df.columns = df.columns.str.replace(" ","_").str.lower()

df['processed'] = df['company_profile'].dropna().apply(preprocess_text)
df = df.loc[~df['processed'].isna()]

In [4]:
# TF-IDF Vectorization

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.95)
X = vectorizer.fit_transform(df['processed']).toarray()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Sentence Transformer

from sentence_transformers import SentenceTransformer

st = time.time()

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
df['encode_transforemers'] = df['processed'].apply(lambda text: model.encode(text, convert_to_numpy=True).flatten())

et = time.time()

print("Elapsed time: {:.2f} seconds".format(et - st))


X_transformers = np.vstack(df['encode_transforemers'])

Elapsed time: 7.22 seconds
0      [0.28014, -0.092356905, -0.21785164, 0.0128366...
1      [-0.27555105, 0.18544693, -0.22933882, 0.20203...
2      [-0.14847693, 0.19307661, -0.23836222, -0.1471...
3      [-0.29696453, 0.31308627, -0.01891439, -0.3029...
4      [-0.34786502, -0.08944182, -0.08626617, -0.171...
                             ...                        
236    [-0.23933008, 0.15753934, -0.08193765, -0.2655...
237    [-0.0896669, 0.47513226, 0.001156422, -0.31884...
238    [0.084984116, -0.5881146, -0.540023, -0.230330...
239    [-0.27149543, -0.099336624, -0.26949438, -0.17...
240    [-0.04237099, 0.40812543, -0.19858725, -0.3898...
Name: encode_transforemers, Length: 238, dtype: object


In [6]:
# GloVe

from torchtext.vocab import GloVe

embeddings = GloVe(name='6B', dim=100)

# Set the maximum sentence length and embedding dimension
max_length = 100 
embedding_dim = 100

# define a function to convert a sentence to a fixed-size vector using GloVe embeddings
def sentence_embedding(sentence):
    words = sentence.split()
    num_words = min(len(words), max_length)
    embedding_sentence = np.zeros((max_length, embedding_dim))
    
    for i in range(num_words):
        word = words[i]
        if word in embeddings.stoi:
            embedding_sentence[i] = embeddings.vectors[embeddings.stoi[word]]
            
    return embedding_sentence.flatten()

df['encode_glove'] = df['processed'].apply(lambda sentence: sentence_embedding(sentence))
X_glove = np.vstack(df['encode_glove'])

/home/hitmonlee/miniconda3/envs/nlp/lib/python3.11/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/home/hitmonlee/miniconda3/envs/nlp/lib/python3.11/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/tmp/ipykernel_15973/2997349109.py:20: DeprecationWarning: __array__ implementation doesn't accept a copy k

In [7]:
# BERT-cls

from transformers import BertTokenizer
from transformers import BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_cls_sentence(sentence):
    # Tokenize input sentence and convert to tensor
    input_ids = torch.tensor([tokenizer.encode(sentence, add_special_tokens=True, max_length=512)])

    # Pass input through BERT model and extract embeddings for [CLS] token
    with torch.no_grad():
        outputs = model(input_ids)
        cls_embedding = outputs[0][:, 0, :]
    
    return cls_embedding.flatten()

In [8]:
st = time.time()

df['cls_bert'] = df['processed'].apply(lambda sentence: get_cls_sentence(sentence))

et = time.time()

print("Elapsed time: {:.2f} seconds".format(et - st))

X_cls_bert = np.vstack(df['cls_bert'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Elapsed time: 15.00 seconds
